In [1]:
## import the tools
import torch
import glob
from pathlib import Path
import os
import numpy as np
from sklearn.neighbors import NearestNeighbors as NN
import glob
import pandas as pd
import ipdb
import laspy
import open3d as o3d
from tqdm.notebook import tqdm
import numpy as np


import open3d as o3d
import laspy


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
### TEST DATA FILENAMES 
#VAL FILES
filename = "PUNKTSKY_00005_1km_6220_495" # OKAY EVEN THO IT IS SRAIGHT LINE
filename = "PUNKTSKY_00005_1km_6221_452" # REALLY BAD EVEN THO IT IS SRAIGHT LINE
# TEST FILES
filename = "PUNKTSKY_00005_1km_6219_494" 
filename = "PUNKTSKY_00005_1km_6211_474" 


In [3]:
# eval_file = "/home/jfriis/msc_data/eval/"+filename+".laz"
# data_val_path_file = "/home/jf/data/denmark/raw/val/"+filename+".laz"
path_to_file = "/home/jf/data/denmark/raw/test/"+filename+".laz"
path = "/home/jf/data/denmark/raw/"

In [12]:
total_points = 0
total_pl_points = 0
amount_of_files = 0
dirs = list(glob.glob(path+"*"))
for dir in dirs:
    files = list(glob.glob(dir+"/*.l*"))
    for file in files:
        data = laspy.read(file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
        label_data = data[data.classification == 14]
        amount_of_points = len(data)
        amount_of_powerline_points = len(label_data)
        total_points += amount_of_points
        total_pl_points += amount_of_powerline_points
        amount_of_files += 1
        print(file.split('/')[-1].replace(".laz",''))
        print(f"Amount of points {amount_of_points} amount of powerline points {amount_of_powerline_points} percentage of interseting points {amount_of_powerline_points/amount_of_points * 100 :.4f}")

print(f"Average amount of points: {total_points/amount_of_files:.0f}. Average amount of pl points: {total_pl_points/amount_of_files:.0f}. Average percentage of pl points: {total_pl_points/total_points * 100 :.4f}")


4095
PUNKTSKY_00005_1km_6219_494
Amount of points 19684629 amount of powerline points 18363 percentage of interseting points 0.0933
4095
PUNKTSKY_00005_1km_6211_474
Amount of points 15599105 amount of powerline points 31418 percentage of interseting points 0.2014
4095
PUNKTSKY_00005_1km_6205_513
Amount of points 21434582 amount of powerline points 63916 percentage of interseting points 0.2982
4095
PUNKTSKY_00005_1km_6168_458
Amount of points 15039646 amount of powerline points 15382 percentage of interseting points 0.1023
4095
PUNKTSKY_00005_1km_6161_465
Amount of points 19994572 amount of powerline points 7401 percentage of interseting points 0.0370
4095
PUNKTSKY_00005_1km_6167_473
Amount of points 33759124 amount of powerline points 57666 percentage of interseting points 0.1708
4095
PUNKTSKY_00005_1km_6146_468
Amount of points 15554025 amount of powerline points 79383 percentage of interseting points 0.5104
4095
PUNKTSKY_00005_1km_6167_474
Amount of points 37849597 amount of powerlin

In [25]:
data = laspy.read(path_to_file, laz_backend=laspy.compression.LazBackend.LazrsParallel)
all_point_data = np.stack([data.X, data.Y, data.Z], axis=0).transpose((1, 0))
print(all_point_data.shape)

4095
(15599105, 3)


In [8]:
def voxelization(point_cloud, voxel_size):
    # Calculate the minimum and maximum coordinates of the point cloud
    min_coord = np.min(point_cloud, axis=0)
    max_coord = np.max(point_cloud, axis=0)

    # Calculate the number of voxels in each dimension
    voxel_counts = ((max_coord - min_coord) / voxel_size).astype(int) + 1

    # Create an empty voxel grid
    voxel_grid = np.zeros(voxel_counts, dtype=int)

    # Assign points to their respective voxels
    voxel_indices = ((point_cloud - min_coord) / voxel_size).astype(int)
    voxel_grid[tuple(voxel_indices.T)] = 1

    return voxel_grid


In [9]:
def count_voxels(voxel_grid):
    num_empty_voxels = np.count_nonzero(voxel_grid == 0)
    num_occupied_voxels = np.count_nonzero(voxel_grid == 1)
    
    return num_empty_voxels, num_occupied_voxels


In [10]:
def calculate_sparsity(voxel_grid):
    num_empty_voxels, num_occupied_voxels = count_voxels(voxel_grid)
    total_voxels = voxel_grid.size

    sparsity = num_empty_voxels / total_voxels

    return sparsity


In [11]:
def calculate_point_cloud_density(point_cloud):
    # Count the number of points
    num_points = len(point_cloud)

    # Calculate the volume (e.g., using the bounding box)
    min_coords = np.min(point_cloud, axis=0)
    max_coords = np.max(point_cloud, axis=0)
    volume = np.prod(max_coords - min_coords)

    # Calculate the density
    density = num_points / volume

    return density

In [12]:
# Assuming you have a point cloud represented as a numpy array
point_cloud = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

# Calculate the density
density = calculate_point_cloud_density(all_point_data)
print("Density:", density)


Density: 1.0815437148998128e-07


In [17]:
# Assuming you have a point cloud represented as a numpy array
point_cloud = np.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]])

# Voxelization
voxel_size = 0.1
voxel_grid = voxelization(all_point_data, voxel_size)

# Calculate sparsity
sparsity = calculate_sparsity(voxel_grid)
print("Sparsity:", sparsity)


MemoryError: Unable to allocate 1.00 EiB for an array with shape (1000001, 1000001, 144231) and data type int64

In [1]:
def calculate_multiplications_3d_unet():
    # Contracting Path (Encoder)
    contracting_layers = [
        (128, 128, 128, 64),  # Layer 1
        (64, 64, 64, 128),    # Layer 2
        (32, 32, 32, 256),    # Layer 3
        (16, 16, 16, 512)     # Layer 4
    ]

    contracting_multiplications = 0
    for layer in contracting_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 3) * (layer[0] * layer[1] * layer[2] * layer[3])
        contracting_multiplications += layer_multiplications

    # Expanding Path (Decoder)
    expanding_layers = [
        (32, 32, 32, 256),    # Layer 4
        (64, 64, 64, 128),    # Layer 3
        (128, 128, 128, 64),  # Layer 2
        (256, 256, 256, 64)   # Layer 1
    ]

    expanding_multiplications = 0
    for layer in expanding_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 3) * (layer[0] * layer[1] * layer[2] * layer[3])
        expanding_multiplications += layer_multiplications

    # Total number of multiplications
    total_multiplications = contracting_multiplications + expanding_multiplications

    return total_multiplications

# Calculate the total number of multiplications
total_multiplications = calculate_multiplications_3d_unet()
print("Total number of multiplications in the U-Net architecture with 3D input:", total_multiplications)


Total number of multiplications in the U-Net architecture with 3D input: 38560333824


In [2]:
def calculate_multiplications_2d_unet():
    # Contracting Path (Encoder)
    contracting_layers = [
        (128, 128, 64),  # Layer 1
        (64, 64, 128),    # Layer 2
        (32, 32, 256),    # Layer 3
        (16, 16, 512)     # Layer 4
    ]

    contracting_multiplications = 0
    for layer in contracting_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 2) * (layer[0] * layer[1] * layer[2])
        contracting_multiplications += layer_multiplications

    # Expanding Path (Decoder)
    expanding_layers = [
        (32, 32, 256),    # Layer 4
        (64, 64, 128),    # Layer 3
        (128, 128, 64),  # Layer 2
        (256, 256, 64)   # Layer 1
    ]

    expanding_multiplications = 0
    for layer in expanding_layers:
        kernel_size = 3
        layer_multiplications = (kernel_size ** 2) * (layer[0] * layer[1] * layer[2])
        expanding_multiplications += layer_multiplications

    # Total number of multiplications
    total_multiplications = contracting_multiplications + expanding_multiplications

    return total_multiplications

# Calculate the total number of multiplications
total_multiplications = calculate_multiplications_2d_unet()
print("Total number of multiplications in the U-Net architecture with 2D input:", total_multiplications)


Total number of multiplications in the U-Net architecture with 2D input: 71958528
